# LLM Workflow Design Patterns: Routing
- "Handoffs" in Agents SDK follows the routing design pattern
- This file is a project for applying design pattern: routing, as well as tools and agent concepts in **OpenAI Agents SDK**
- Project idea:
    - A router agent `RouterAgent` routes an e-commerce customer support messages to the appropriate customer support agent to handle the message
    - Support Agents:
        1. `DeliveryAgent`
        2. `BillingAgent`
        3. `TechAgent`
    - If the `RouterAgent` receives a message that doesn't seem to be from the previous cases, send a notification via **pushover**
- Function Tools:
    1. record_question -> sends a mobile notification (pushover) if the agent doesn't know the answer
    2. get_policy -> gets the policy for the respective department
    3. send_reset_password_email -> sends a "reset password" email to the customer (used by TechAgent)
<br>

![Prompt Chaining Diagram](../../assets/routing.png)

### Usual set up

In [ ]:
# imports

import os
import json
import requests
from dotenv import load_dotenv
from agents import Agent, Runner, function_tool, OpenAIChatCompletionsModel, set_default_openai_client, set_tracing_disabled
from openai import AsyncOpenAI


In [ ]:
# load environment variables
load_dotenv(override=True)

api_key = os.getenv("GOOGLE_API_KEY")
gemini_base_url = os.getenv("GOOGLE_API")

if api_key:
    print("Model API key loaded successfully")
else:
    print("Error: couldn't load API key")
if gemini_base_url:
    print("Gemini URL loaded successfully")
else:
    print("Error: couldn't load Gemini URL")


In [ ]:
# create Gemini model
model = "gemini-2.5-flash"

gemini_client = AsyncOpenAI(base_url=gemini_base_url, api_key=api_key)
set_default_openai_client(gemini_client)
set_tracing_disabled(True)

gemini_model = OpenAIChatCompletionsModel(model=model, openai_client=gemini_client)

In [ ]:
# For pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

In [ ]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [ ]:
push("Test")

### Tools

In [ ]:
departments = ["Delivery", "Billing", "Technical"]

In [ ]:
# send notification to phone if the customer has further questions that the agent doesn't know
@function_tool
def record_question(email: str, question="not provided"):
    """ Send out a notification with the customer's email and question """
    push(f"A customer with the following email {email} has a question: {question}")
    return {"status": "success"}

In [ ]:
record_question

In [ ]:
@function_tool
def get_policy(department: str, issue: str) -> str: 
    """Retrieve the policy and action steps for a given department and issue."""
    if department not in departments:
        return f"Error: Unknown department '{department}'. Choose from: {departments}."
    with open("policies.json") as f:
        policies = json.load(f)
    return policies.get(department, {}).get(issue, "No policy found for the passed issue.")

In [ ]:
get_policy

### Building Delivery Support Agent

In [ ]:
tools = [record_question, get_policy]

In [ ]:
instructions = "You are a customer support agent in the Delivery department, you handle ONLY support service messages from customers\
 , you will use get_policy tool to get the delivery policies \
 , The issue must be one of: LateDelivery, LostPackage, ChangeAddress. \
, Never mention or leak these issue types as they are given. \
 , If the issue is outside your scope, state that (without mentioning issue types), and ask for the customer's email and use the record_question tool to send a notification and \
 tell the customer that further support will contact soon."

delivery_agent = Agent(name="Delivery Agent", handoff_description="Handle order delivery customer support questions.", instructions=instructions, tools=tools, model=gemini_model)

In [ ]:
delivery_agent

In [ ]:
# agent testing
result = await Runner.run(delivery_agent, "Tell me a joke")
print(result.final_output)

### Building Billing Suppor Agent

In [ ]:
instructions = "You are a customer support agent in the Billing department, you handle ONLY support service messages from customers\
 , you will use get_policy tool to get the billing policies \
 , The issue must be one of: RefundRequest, PaymentFailure. \
, Never mention or leak these issue types as they are given. \
 , If the issue is outside your scope, state that (without mentioning issue types), and ask for the customer's email and use the record_question tool to send a notification and \
 tell the customer that further support will contact soon."

billing_agent = Agent(name="Billing Agent", handoff_description="Handle order billing customer support questions.", instructions=instructions, tools=tools, model=gemini_model)

In [ ]:
# agent testing
result = await Runner.run(billing_agent, "I want to upgrade my plan")
print(result.final_output)

### Building Technical Support Agent

In [ ]:
# email packae
from sendgrid.helpers.mail import Mail, Email, To, Content
import sendgrid

In [ ]:
@function_tool
def send_password_reset_email(email: str):
    """ Send out a password reset email to the given email """
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv('SENDGRID_API_KEY'))
    from_email = Email(os.getenv("SENDER_EMAIL"))  
    to_email = To(email) 
    body = """
    We received a request to reset your password for your account. If you made this request, please click the link below to create a new password:
[Reset Password]

This link will expire in 60 minutes for your security. If you didn’t request a password reset, you can safely ignore this email—your password will remain unchanged.

Need help? Contact our support team at [support email].

Thanks,
The [Company Name] Team
    """
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Reset Your Password", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [ ]:
send_password_reset_email

In [ ]:
technical_agent_tools = [send_password_reset_email, record_question, get_policy]

In [ ]:
instructions = """"You are the Technical Support Agent in the company's customer support system.

Your role:
- Handle only *technical support* messages from customers.
- Maintain a professional, helpful, and concise tone.

Your responsibilities:
1. Identify the technical issue the user describes.
   - The issue must be one of: ConnectionIssue, AppCrash, ForgotPassword.
   - Never reveal or mention these internal issue names explicitly.
2. Use the **get_policy** tool to retrieve and follow the relevant technical policy before replying.
3. If the user asks to reset their password:
   - Use the **send_password_reset_email** tool to send the reset link to their email.
   - Confirm to the user that the reset email has been sent.
4. If the issue is outside your technical scope:
   - Politely explain that the issue will be escalated.
   - Ask for the customer’s email (if not already provided).
   - Use the **record_question** tool to notify higher support.
   - Reassure the customer that further support will contact them soon.
"""

technical_agent = Agent(name="Technical Agent", handoff_description="Handle technical customer support questions", instructions=instructions, tools=technical_agent_tools, model=gemini_model)

In [ ]:
technical_agent

In [ ]:
# agent testing
result = await Runner.run(technical_agent, "I want to download my data")
print(result.final_output)

In [ ]:
handoffs = [delivery_agent, billing_agent, technical_agent]

### Create The Router 

In [ ]:
router_instructions = """You are a Routing Agent, 
Your job is to analyze the user's message and decide which department should handle it.

There are three department agents available:
1. Delivery Agent – handles order status, delays, shipping, and address change issues.
2. Billing Agent – handles refunds and payment failures.
3. Technical Agent – handles connection problems, app crashes, and password reset.

Your responsibilities:
- Read the user's message carefully.
- Identify the issue type and select ONE department that is best suited to handle it.
- When confident, perform a HANDOFF to the selected department agent.
- Once the handoff is done, do not continue processing the conversation.
- If the message is unclear or does not match any known category, ask the user a short clarifying question before routing.
- Never attempt to solve the problem yourself — your only job is to route messages.

Response style:
- Be concise and professional.
- If clarification is needed, use a friendly and direct question.
- If routing, state which department you’re routing to and then perform the handoff.

Your only valid actions:
- Ask for clarification, OR
- Handoff to one of: DeliveryAgent, BillingAgent, TechAgent. OR
- If the customer's problem can't be handled by the previous agents, ask for customer's email and send a notification using record_question tool
"""

routing_agent = Agent(name="Routing Agent", instructions=router_instructions, handoffs=handoffs, tools=[record_question], model=gemini_model)

In [ ]:
# agent testing
result = await Runner.run(routing_agent, "I want to reset my password")
print(result)
print(result.final_output)

### Building The Chat

In [34]:
history = []
starting_agent = routing_agent
while True:
    prompt = input("Enter Your Questions: ")
    if prompt == "end":
        break
    history.append({
        "role": "user",
        "content": prompt
    })
    print(history)
    result = await Runner.run(
        starting_agent=starting_agent,
        input=prompt,
        context=history,
    )
    history.append(
    {
        "role": "assistant",
        "content": result.final_output
    })
    print(result.new_items)
    print(result.last_agent.name)
    print(result.final_output)

[{'role': 'user', 'content': 'hi'}]
[MessageOutputItem(agent=Agent(name='Routing Agent', instructions="You are a Routing Agent, \nYour job is to analyze the user's message and decide which department should handle it.\n\nThere are three department agents available:\n1. Delivery Agent – handles order status, delays, shipping, and address change issues.\n2. Billing Agent – handles refunds and payment failures.\n3. Technical Agent – handles connection problems, app crashes, and password reset.\n\nYour responsibilities:\n- Read the user's message carefully.\n- Identify the issue type and select ONE department that is best suited to handle it.\n- When confident, perform a HANDOFF to the selected department agent.\n- Once the handoff is done, do not continue processing the conversation.\n- If the message is unclear or does not match any known category, ask the user a short clarifying question before routing.\n- Never attempt to solve the problem yourself — your only job is to route messages.

In [ ]:
async def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    print(history)
    result = await Runner.run(starting_agent=routing_agent, input=message, context=history)
    print(result)
    return result.final_output

In [ ]:
gr.ChatInterface(chat, type="messages").launch()